<link rel="stylesheet" href="../../styles/theme_style.css">
<!--link rel="stylesheet" href="../../styles/header_style.css"-->
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">

<table width="100%">
    <tr>
        <td id="image_td" width="15%" class="header_image_color_1"><div id="image_img"
        class="header_image_15"></div></td>
        <td class="header_text"> Synchronising Android and PLUX sensors</td>
    </tr>
</table>

<div id="flex-container">
    <div id="diff_level" class="flex-item">
        <strong>Difficulty Level:</strong>   <span class="fa fa-star checked"></span>
                                <span class="fa fa-star checked"></span>
                                <span class="fa fa-star checked"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
    </div>
    <div id="tag" class="flex-item-tag">
        <span id="tag_list">
            <table id="tag_list_table">
                <tr>
                    <td class="shield_left">Tags</td>
                    <td class="shield_right" id="tags">other&#9729;android&#9729;opensignals mobile&#9729;bitalino</td>
                </tr>
            </table>
        </span>
        <!-- [OR] Visit https://img.shields.io in order to create a tag badge-->
    </div>
</div>

The <strong><span class="color2">OpenSignals mobile application</span></strong> allows to acquire data from internal Android phone sensors and PLUX sensors at the same time. In order to put these into a proper context, the signals acquired from all these sensors need to be synchronised.

In this <strong><span class="color13">Jupyter notebook</span></strong> we will show you the essential steps in order to achieve this. First, we will be doing some minor data manipulation. This is then followed by having a look at the data we want to synchronise. Finally, all signals will be synchronised and written into one file.

The Android data used in this notebook was recorded using a <strong>Samsung Galaxy A40</strong> with a set sampling rate of <strong>100 Hz</strong>. We recorded data from the <strong>accelerometer</strong>, <strong>GPS</strong>, <strong>linear accelerometer</strong>, and <strong>rotation vector</strong> sensors. Prior to starting this notebook, we already did a synchronisation of the Android sensors. All sensors were resampled to <strong>100 Hz</strong> using a padding of type 'same' and an interpolation using the 'previous' type. If you want to find out how this can be done, please have a look at our <a href="android_signal_resampling.ipynb">Resampling of signals recorded with Android sensors <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a> notebook  on Android file synchronisation.

The other data was acquired using a new, fresh out of the box, <span class="color4"><strong>BITalino</strong></span> with a sampling rate of <strong>100 Hz</strong>. We recorded data from the accelerometer sensor. Since the <span class="color4"><strong>BITalino</strong></span> was not altered before acquiring data, only the z-axis of the accelerometer was available. In case you choose to make an acquisition with your PLUX device with a sampling rate of 1000 Hz, you should resample your Android signals to the same sampling rate.

To synchronise both device we took the same approach presented in the <a href="../Pre-Process/synchrony_acc.ipynb">notebook <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a> on synchronisation of two PLUX devices  using the accelerometers. At the beginning of our recording we produced a clearly distinctive event on both accelerometers by moving the devices along their z-axes. The z-axis of the Android phone is perpendicular to the screen.
    
In case this is your first time working with Android sensors, we highly recommend reading the <a href="intro_to_android_sensors.ipynb">Introduction to Android sensors <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a> notebook with general information on Android sensors.

<hr>

<p class="steps">1 - Package imports</p>
First, let's import <strong><span class="color2">biosignalsnotebooks</span></strong> package and some other packages. All functionalities that we need are within these packages.

In [2]:
# biosignalsnotebooks package
import biosignalsnotebooks as bsnb

# numpy packaged
from numpy import loadtxt

# package for using operating system dependent functionality
import os

# needed in order to extract information from the header in a convenient way
import json

<p class="steps">2 - Unit conversion of the BITalino signals</p>
If you want to have a file in which all values are physical quantities, then it makes sense to convert the RAW values acquired with the <span class="color4"><strong>BITalino</strong></span>. We are also going to change the axis indicating the number of samples to a real time axis. This is done, so that in the final synchronised file, both the <span class="color4"><strong>BITalino</strong></span> and the Android sensors use time axis. However, this step can also be skipped if you are not in need of that. We will convert the values and subsequently save them to a new file. For value conversion we can make use of the <a href="https://github.com/biosignalsplux/biosignalsnotebooks/blob/f70aba9c31a4f759ede2759a8f6cdc6340ec3a21/biosignalsnotebooks/biosignalsnotebooks/conversion.py#L31-L319">raw_to_phy(...) <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a> function of the <strong><span class="color2">biosignalsnotebooks</span></strong> package. 


The function takes the following inputs:

<ul>
    <li><span class="color1"><strong>sensor (string):</strong></span> The name of the sensor as string. Supported sensors are 'TEMP',  'EMG', 'ECG', 'BVP', 'SpO2.ARM', 'SpO2.HEAD', 'SpO2.FING', 'ACC', 'EEG', and 'EDA'. </li><br>
    <li><span class="color2"><strong>device (string):</strong></span> The name of the device as string.</li><br>
    <li><span class="color1"><strong>raw_signal (array):</strong></span> The raw signal data to be converted.</li><br>
    <li><span class="color2"><strong>resolution (int):</strong></span> The resolution of the sensor.</li><br>
    <li><span class="color1"><strong>option (string):</strong></span><br> A abbreviation for the physical value to which the raw signal should be converted to. Depending on the type of signals, the following options are available: 'Ohm', 'K', 'C' (for 'TEMP') | 'mV', 'V' (for 'EMG' and 'ECG') | 'uA', 'A' (for 'BVP', 'SpO2.ARM', 'SpO2.HEAD', and 'SpO2.FING') | 'g' (for 'ACC') | 'uV', 'V' (for 'EEG) | 'uS', 'S' (for 'EDA') </li>
</ul>

But first, we need to load the raw signal file, open it, and extract all needed information from the header.

<p class="steps">2.1 - Get metadata through the read of file header</p>

In [3]:
# set file path
path = '../../images/other/android_bitalino_sync/opensignals_98D351FD8E78_2020-07-29_19-51-58.txt'
     
# get the header of the file
with open(path, encoding='latin-1') as opened_file:
    # read the information from the header lines (omitting the begin and end tags of the header)
    header = opened_file.readlines()[1][2:]  # omit "# " at the beginning of the sensor information
    
    # convert the header to a dictionary (for easier access of the information that we need)
    header = json.loads(header)

<p class="steps">2.2 - Access/store relevant metadata about the sensor, device and acquisition</p>

In [4]:
# get the key first key of the dictionary (all information can be accessed using that key)
dict_key = list(header.keys())[0]
    
# get the values for unit conversion
device = header[dict_key]['device']
resolution = header[dict_key]['resolution'][0]

# get the sampling rate of the device
sampling_rate = header[dict_key]['sampling rate']

# set the sensor name and the option
sensor_name = 'ACC'
option = 'm/s^2'

<p class="steps">2.3 - Load BITalino data from file</p>

In [5]:
# get the data from the file (as an numpy array) 
data = loadtxt(path)

<p class="steps">2.4 - Generate a time-axis and convert RAW values to acceleration physical units</p>

In [6]:
# convert the sampling axis to a time axis
data[:, 0] = bsnb.generate_time(data[:, -1], sampling_rate)

# convert the data of the ACC channel (in our case the ACC data is the last column in the file. This can be accessed using -1)
# in order to make things simple we are directly overwriting that channel in the data array
data[:, -1] = bsnb.raw_to_phy(sensor_name, device, data[:, -1], resolution, option)

<p class="steps">2.5 - Creation of a new file with the converted time and signal values</p>

In [7]:
# set file name for the file in which the converted data is going to be saved
file_name = 'bitalino_converted.txt'

# get the current directory
curr_path = os.path.abspath(os.getcwd())

# crete full file path
save_path = os.path.join(curr_path, file_name)

# convert header to string
header_string = "# OpenSignals Text File Format\n# " + json.dumps(header) + '\n# EndOfHeader\n'

# open a new file at the path location
sync_file = open(save_path, 'w')

# write the header to the file
sync_file.write(header_string)

# write the data to the file. The values in each line are written tab separated
for row in data:
    sync_file.write('\t'.join(str(value) for value in row) + '\t\n')

# close the file
sync_file.close()

<p class="steps">3 - Taking a look at the channels used for synchronisation</p>
Before we start synchronising both files, we will take a look at both accelerometer channels that we are going to use for synchronisation. The z-axis of the accelerometer are the fifth ('CH5') and the second channel ('CH2') for the <span class="color4"><strong>BITalino</strong></span> and the Android sensor files, respectively.

As we can see in the plot below, both channels are only slightly shifted in relation to each other. This shift may however change from recording to recording, because of the Android system's sampling procedure. In case you are wondering: The Android accelerometer sensor measures the acceleration <strong>including</strong> the force of gravity, thus the data is shifted along the y-axis of the plot.

In [8]:
# set file path
path = '../../images/other/android_bitalino_sync/'

# set full file paths
# for the BITalino we reuse the path where we saved the file
bitalino_file = save_path
android_file =  path + 'android_sensor_sync.txt'

# load the data
bitalino_data = bsnb.load(bitalino_file, get_header=False)
android_data = bsnb.load(android_file, get_header=False)

# plot data
bitalino_time = loadtxt(bitalino_file)[:, 0]
android_time = loadtxt(android_file)[:, 0]


# plot the data of both files
bsnb.plot([bitalino_time, android_time], [bitalino_data['CH5'], android_data['CH2']], legend_label=["bitalino CH5", "android CH2"], y_axis_label=["Accelerometer", "Accelerometer"], x_axis_label="Time (s)")

<p class="steps">4 - Create the synchronised file</p>
Now, that we have everything set up, we can perform the synchronisation of the two files. This can be achieved by calling the 
<a href="https://github.com/biosignalsplux/biosignalsnotebooks/blob/e6391dc3cc55fc69dc7cb5b7a6bf66a90ccf7e02/biosignalsnotebooks/biosignalsnotebooks/synchronisation.py#L143-L175">generate_sync_txt_file(...) <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a> function of the <strong><span class="color2">biosignalsnotebooks</span></strong> package. 

The function takes the following inputs:

<ul>
    <li><span class="color4"><strong>in_path (string or list of strings):</strong></span> A path or a list of paths, as string, pointing to the files that are supposed to be synchronised. If the input is a string, it is assumed that the two signals are in the same file, else, if the input is a list,
        it is assumed that the two signals are in different file. </li><br>
    <li><span class="color6"><strong>channels (list):</strong></span> List with the strings identifying the channels of each signal. (default: ("CH1", "CH1")).</li><br>
    <li><span class="color4"><strong>new_path (string):</strong></span> The path to create the new file. (default: 'sync_file.txt')</li>
</ul>

In [9]:
# put list in file
input_paths = [bitalino_file, android_file]

# set path, including new file name
new_path = os.path.join(curr_path, 'android_bitalino_sync.txt')

# synchronize
bsnb.generate_sync_txt_file(input_paths, channels=('CH5', 'CH2'), new_path=new_path)

In this <strong><span class="color4">Jupyter notebook</span></strong> we learned how to synchronise data acquired from PLUX and Android sensor at he same time using the <strong><span class="color2">biosignalsnotebooks</span></strong> package.

<strong><span class="color7">We hope that you have enjoyed this guide</span></strong>. <strong><span class="color2">biosignalsnotebooks</span> <span class="color4"> is an environment in continuous expansion, so don't stop your journey and learn more with the remaining</span> 
<a href="https://biosignalsplux.com/learn/notebooks.html">Notebooks <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a></strong>.


<span class="color6"><strong>Auxiliary Code Segment (should not be replicated by
the user)</strong></span>

In [1]:

from biosignalsnotebooks.__notebook_support__ import css_style_apply
css_style_apply()

.................... CSS Style Applied to Jupyter Notebook .........................


In [11]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'],
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>